In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/splits/brain_split/val/mr/1BA305_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BA247_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BC056_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BB151_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BB066_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BC053_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BB071_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BC073_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BC082_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BA005_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BA239_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BC077_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BC088_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BA358_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BC039_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BB030_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BB182_mr.nii
/kaggle/input/splits/brain_split/val/mr/1BC051_mr.nii
/kaggle/input/splits/brain_s

In [2]:
import nibabel as nib
import numpy as np
import os
import random
import shutil
from pathlib import Path
from scipy.ndimage import affine_transform

# Define paths
split_dir = Path("/kaggle/input/splits/brain_split/train")
output_root = Path("/kaggle/working/train_transformed")

# Reset output directory
if output_root.exists():
    shutil.rmtree(output_root)
output_root.mkdir(parents=True, exist_ok=True)

for subfolder in ["ct", "mr", "mask"]:
    (output_root / subfolder).mkdir(parents=True, exist_ok=True)

# Transformation parameters
def generate_affine_matrix():
    angle = random.uniform(-5, 5) * np.pi / 180  # rotate max 5 degrees
    tx = random.uniform(-5, 5)  # translate max 5 pixels
    ty = random.uniform(-5, 5)

    cos_theta = np.cos(angle)
    sin_theta = np.sin(angle)
    
    matrix = np.array([
        [cos_theta, -sin_theta],
        [sin_theta,  cos_theta]
    ])
    offset = [tx, ty]

    return matrix, offset

# Get all CTs
ct_files = list((split_dir / "ct").glob("*.nii"))

# Shuffle for randomness
random.shuffle(ct_files)

# Transform 80%
num_transform = int(0.8 * len(ct_files))
transformed_cts = ct_files[:num_transform]
remaining_cts = ct_files[num_transform:]

# Apply transformations
for ct_path in transformed_cts:
    file_id = ct_path.stem.replace("_ct", "")
    mr_path = split_dir / "mr" / f"{file_id}_mr.nii"
    mask_path = split_dir / "mask" / f"{file_id}_mask.nii"

    ct_img = nib.load(ct_path)
    ct_data = ct_img.get_fdata()

    matrix, offset = generate_affine_matrix()

    transformed_data = np.zeros_like(ct_data)
    for i in range(ct_data.shape[2]):
        transformed_data[:, :, i] = affine_transform(
            ct_data[:, :, i], matrix, offset=offset, order=1, mode='nearest'
        )

    transformed_img = nib.Nifti1Image(transformed_data, ct_img.affine)
    nib.save(transformed_img, output_root / "ct" / f"{file_id}_ct.nii")

    shutil.copy(mr_path, output_root / "mr" / f"{file_id}_mr.nii")
    if mask_path.exists():
        shutil.copy(mask_path, output_root / "mask" / f"{file_id}_mask.nii")

# Copy remaining aligned CTs
for ct_path in remaining_cts:
    file_id = ct_path.stem.replace("_ct", "")
    mr_path = split_dir / "mr" / f"{file_id}_mr.nii"
    mask_path = split_dir / "mask" / f"{file_id}_mask.nii"

    shutil.copy(ct_path, output_root / "ct" / f"{file_id}_ct.nii")
    shutil.copy(mr_path, output_root / "mr" / f"{file_id}_mr.nii")
    if mask_path.exists():
        shutil.copy(mask_path, output_root / "mask" / f"{file_id}_mask.nii")

print(f"✅ Done! {len(transformed_cts)} CTs transformed (slice-by-slice), {len(remaining_cts)} kept original.")


✅ Done! 115 CTs transformed (slice-by-slice), 29 kept original.
